In [19]:
import os 
import json
import pandas as pd
import traceback

In [20]:
from langchain.chat_models import ChatOpenAI

In [21]:
from dotenv import load_dotenv
load_dotenv() #take environment variables from .env 

True

In [22]:
# First writedown your own OPENAI API KEY
KEY = os.getenv('OPENAI_API_KEY')

In [23]:
llm = ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature='0.5')

In [24]:
from langchain .llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [25]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [26]:
TEMPLATE="""
Text{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sute the questions are not repeated and check all the questions to be confirmed the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [27]:
quize_generation_prompt = PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [28]:
quiz_chain = LLMChain(llm=llm, prompt=quize_generation_prompt, output_key='quiz', verbose=True)

In [29]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [30]:
quize_evaluation_prompt = PromptTemplate(
    input_variables=['subject','quiz'], 
    template=TEMPLATE2
)

In [31]:
review_chain = LLMChain(llm=llm, prompt=quize_evaluation_prompt, output_key='review', verbose=True)

In [32]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain], 
    input_variables=['text','number','subject','tone','response_json'],
    output_variables=['quiz','review'],
    verbose=True
)

In [61]:
#reading the data.txt 
file_path = r'C:\Users\UpforceTech\mcqgen\data.txt'

with open(file_path,'r') as file:
    TEXT = file.read()

print(TEXT)

Globalization is a process that encompasses the causes, courses, and consequences of transnational and transcultural integration of human and non-human activities. India had the distinction of being the world's largest economy till the end of the Mughal era, as it accounted for about 32.9% share of world GDP and about 17% of the world population. The goods produced in India had long been exported to far off destinations across the world;[1] the concept of globalization is hardly new to India.

India currently accounts for 2.7% of world trade (as of 2015), up from 1.2% in 2006 according to the World Trade Organization (WTO).[2] Until the liberalisation of 1991, India was largely and intentionally isolated from the world markets, to protect its fledgeling economy and to achieve self-reliance. Foreign trade was subject to import tariffs, export taxes and quantitative restrictions, while foreign direct investment was restricted by upper-limit equity participation, restrictions on technolog

In [62]:
# Conveting the python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [63]:
NUMBER = 5
SUBJECT ='globalization in india'
TONE = 'simple'

In [64]:
# get_openai_callback() is used to set up token usage tracking in langchain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            'text':TEXT,
            'number':NUMBER,
            'subject':SUBJECT,
            'tone':TONE,
            'response_json': json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

TextGlobalization is a process that encompasses the causes, courses, and consequences of transnational and transcultural integration of human and non-human activities. India had the distinction of being the world's largest economy till the end of the Mughal era, as it accounted for about 32.9% share of world GDP and about 17% of the world population. The goods produced in India had long been exported to far off destinations across the world;[1] the concept of globalization is hardly new to India.

India currently accounts for 2.7% of world trade (as of 2015), up from 1.2% in 2006 according to the World Trade Organization (WTO).[2] Until the liberalisation of 1991, India was largely and intentionally isolated from the world markets, to protect its fledgeling economy and to achieve self-reliance. Foreign trade was subject to import tariffs, export taxes and quantitative restrictions, wh

In [65]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:3853
Prompt Tokens:3476
Completion Tokens:377
Total Cost:0.005968


In [66]:
quiz = response.get('quiz')
quiz = json.loads(quiz)

In [67]:
quiz

{'1': {'mcq': 'What is the current percentage of world trade that India accounts for?',
  'options': {'a': '1.2%', 'b': '2.7%', 'c': '7.7%', 'd': '9.5%'},
  'correct': 'b'},
 '2': {'mcq': "What was the value of India's international trade in 2003-04?",
  'options': {'a': '$12.50 billion',
   'b': '$63,0801 billion',
   'c': '$17.68 billion',
   'd': '$12.31 billion'},
  'correct': 'b'},
 '3': {'mcq': "Which countries are India's trading partners?",
  'options': {'a': 'China, US, UAE, UK, Japan, EU',
   'b': 'Brazil, China, Russia',
   'c': 'Mauritius, Singapore, United Kingdom, Japan, United States',
   'd': 'China, US, UK, Japan, EU'},
  'correct': 'a'},
 '4': {'mcq': 'What is the current percentage of FDI in India compared to GDP?',
  'options': {'a': '0.1%', 'b': '2%', 'c': '7.76%', 'd': '6.25%'},
  'correct': 'b'},
 '5': {'mcq': 'What is the leading source of remittances to India?',
  'options': {'a': 'Indian workers employed outside the country',
   'b': 'Foreign direct investment

In [68]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [69]:
quiz_table_data

[{'MCQ': 'What is the current percentage of world trade that India accounts for?',
  'Choices': 'a: 1.2% | b: 2.7% | c: 7.7% | d: 9.5%',
  'Correct': 'b'},
 {'MCQ': "What was the value of India's international trade in 2003-04?",
  'Choices': 'a: $12.50 billion | b: $63,0801 billion | c: $17.68 billion | d: $12.31 billion',
  'Correct': 'b'},
 {'MCQ': "Which countries are India's trading partners?",
  'Choices': 'a: China, US, UAE, UK, Japan, EU | b: Brazil, China, Russia | c: Mauritius, Singapore, United Kingdom, Japan, United States | d: China, US, UK, Japan, EU',
  'Correct': 'a'},
 {'MCQ': 'What is the current percentage of FDI in India compared to GDP?',
  'Choices': 'a: 0.1% | b: 2% | c: 7.76% | d: 6.25%',
  'Correct': 'b'},
 {'MCQ': 'What is the leading source of remittances to India?',
  'Choices': 'a: Indian workers employed outside the country | b: Foreign direct investment | c: Foreign aid | d: Commercial borrowing',
  'Correct': 'a'}]

In [70]:
quiz = pd.DataFrame(quiz_table_data)

quiz.to_csv('machinelearning.csv', index=False)

In [71]:
from datetime import datetime

In [77]:
LOG_FILE = f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"
LOG_FILE

'01_08_2024_14_33_18.log'